Code Cells realisées par EL-AMIRI Youssef(N°21):
    [1],[2]
Code Cells realisées par DERKAOUI Yahia(N°10):
    [3]

In [1]:
from collections import deque
import copy

In [ ]:
class SudokuCSP:
    def __init__(self, grid):
        self.variables = [(r, c) for r in range(9) for c in range(9)]
        self.domains = {var: set(range(1, 10)) for var in self.variables}
        self.neighbors = {var: set() for var in self.variables}
        self.init_domains(grid)
        self.create_constraints()

    def init_domains(self, grid):
        for r in range(9):
            for c in range(9):
                val = grid[r][c]
                if val != 0:
                    self.domains[(r, c)] = {val}

    def create_constraints(self):
        for r in range(9):
            for c in range(9):
                var = (r, c)
                for i in range(9):
                    if i != c:
                        self.neighbors[var].add((r, i))
                    if i != r:
                        self.neighbors[var].add((i, c))
                box_r, box_c = 3 * (r // 3), 3 * (c // 3)
                for i in range(box_r, box_r + 3):
                    for j in range(box_c, box_c + 3):
                        if (i, j) != var:
                            self.neighbors[var].add((i, j))

    def is_consistent(self, var, value, assignment):
        for neighbor in self.neighbors[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False
        return True

    def ac3(self):
        queue = deque([(xi, xj) for xi in self.variables for xj in self.neighbors[xi]])
        while queue:
            xi, xj = queue.popleft()
            if self.revise(xi, xj):
                if not self.domains[xi]:
                    return False
                for xk in self.neighbors[xi] - {xj}:
                    queue.append((xk, xi))
        return True

    def revise(self, xi, xj):
        revised = False
        for x in set(self.domains[xi]):
            if not any(x != y for y in self.domains[xj]):
                self.domains[xi].remove(x)
                revised = True
        return revised

    def select_unassigned_variable(self, assignment):
        unassigned = [v for v in self.variables if v not in assignment]
        return min(unassigned, key=lambda var: len(self.domains[var]))

    ### backtrack() and solve() methods are done by Yahia not Youssef ###

    def backtrack(self, assignment):
        if len(assignment) == 81:
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in sorted(self.domains[var]):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                backup = copy.deepcopy(self.domains)
                self.domains[var] = {value}
                if self.ac3():
                    result = self.backtrack(assignment)
                    if result:
                        return result
                self.domains = backup
                del assignment[var]
        return None

    def solve(self):
        if not self.ac3():
            return None
        # Vérifie si chaque domaine a exactement une seule valeur
        if all(len(self.domains[var]) == 1 for var in self.variables):
            return {var: next(iter(self.domains[var])) for var in self.variables}
        return None



In [3]:
def print_grid(grid):
    for row in grid:
        print(" ".join(str(val) if val != 0 else '.' for val in row))

def main():
    # Grille Sudoku de l'exercice (exemple — à adapter si besoin)
    grid = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    print("🧩 Grille Sudoku à résoudre :")
    print_grid(grid)

    csp = SudokuCSP(grid)
    solution = csp.solve()

    if solution:
        print("\n✅ Solution trouvée :")
        solved_grid = [[solution[(r, c)] for c in range(9)] for r in range(9)]
        print_grid(solved_grid)
    else:
        print("\n❌ Aucune solution trouvée.")

if __name__ == "__main__":
    main()


🧩 Grille Sudoku à résoudre :
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9

✅ Solution trouvée :
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
